## Benchmarking ToMe
We provide some benchmarking code in order to benchmark ToMe's throughput.

**Note**: notebooks have an overhead. To properly benchmark, use a standalone script.

In [ ]:
import timm
import tome

In [2]:
# Use any ViT model here (see timm.models.vision_transformer)
model_name = "vit_base_patch16_224"

# Load a pretrained model
model = timm.create_model(model_name, pretrained=True)

In [3]:
# Set this to be whatever device you want to benchmark on
# If you don't have a GPU, you can use "cpu" but you probably want to set the # runs to be lower
device = "cuda:0"
runs = 50
batch_size = 256  # Lower this if you don't have that much memory
input_size = model.default_cfg["input_size"]

In [4]:
# Baseline benchmark
baseline_throughput = tome.utils.benchmark(
    model,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size
)

Benchmarking: 100%|██████████| 50/50 [00:16<00:00,  2.98it/s]


Throughput: 518.00 im/s


### Applying ToMe
Simply patch the model after initialization to enable ToMe.

In [5]:
# Apply ToMe
tome.patch.timm(model)

In [8]:
# ToMe with r=16
model.r = 16
tome_throughput = tome.utils.benchmark(
    model,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size
)
print(f"Throughput improvement: {tome_throughput / baseline_throughput:.2f}x")

Benchmarking: 100%|██████████| 50/50 [00:09<00:00,  5.19it/s]


Throughput: 1060.45 im/s
Throughput improvement: 2.05x


In [9]:
# ToMe with r=16 and a decreasing schedule
model.r = (16, -1.0)
tome_decr_throughput = tome.utils.benchmark(
    model,
    device=device,
    verbose=True,
    runs=runs,
    batch_size=batch_size,
    input_size=input_size
)
print(f"Throughput improvement: {tome_decr_throughput / baseline_throughput:.2f}x")

Benchmarking: 100%|██████████| 50/50 [00:06<00:00,  7.91it/s]

Throughput: 1589.12 im/s
Throughput improvement: 3.07x
